<div align=center>

# Linear regression workflow 📈

$$ Y = m X + b $$

|Symbol|Description|
|-|-|
|$Y$|Result|
|$m$|Weights|
|$X$|Function argument|
|$b$|Bias|

</div>

## Check if the kernel is up and working ✨

In [ ]:
print("Kernel works!")

### Import required packages 📦

In [118]:
from safetensors.torch import save_file
from safetensors.torch import safe_open
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import torch

## Setup the workflow ⚒️
<div align=center>

![pytorch_workflow](pytorch_workflow.png)

</div>

### Configuration for optimized computation and plotting

In [119]:
# Set float16 as dtype for more optimized computation
torch.set_default_dtype(torch.half)

# Set the plot figsize
plot_figsize = (12, 8)

# Set the global variables
train_data_color = "green"
val_data_color = "orange"
pred_color = "red"

### 1. Get data ready 🔢

#### Define a plotting functions

In [120]:
# Define data plotting function
def plot_data(train_data, val_data, train_labels, val_labels, title="Data plot", predictions=None):
    
    # Set the font to be JetBrains Mono
    font_path = Path("../fonts/JetBrainsMono-Regular.ttf")

    # Set the dark mode
    plt.style.use('dark_background')

    # Create the plot
    plt.figure(figsize=plot_figsize)
    plt.plot(train_data, train_labels, c=train_data_color, label="Training data")
    plt.scatter(train_data, train_labels, c=train_data_color)

    plt.plot(val_data, val_labels, c=val_data_color, label="Validation data")
    plt.scatter(val_data, val_labels, c=val_data_color)

    if predictions is not None:
        plt.plot(val_data, predictions, c=pred_color, label="Model predictions")
        plt.scatter(val_data, predictions, c=pred_color)

    # Setting up axes
    plt.title(title, font=font_path, fontsize=16)
    plt.xlabel('X', font=font_path, fontsize=14)
    plt.ylabel('Y', font=font_path, fontsize=14)

    # Add grid and axes lines
    plt.axhline(0, color='white', lw=0.5, ls='--')
    plt.axvline(0, color='white', lw=0.5, ls='--')
    plt.grid(color='gray', linestyle='--', linewidth=0.5)

    # Customize ticks
    plt.xticks(font=font_path)
    plt.yticks(font=font_path)

    # Add a legend
    plt.legend(prop=font_path)

In [121]:
# Define loss curves plotting function
def plot_loss_curves(train_loss, val_loss, epoch_count, title="Loss curves plot"):
    
    # Set the font to be JetBrains Mono
    font_path = Path("../fonts/JetBrainsMono-Regular.ttf")

    # Set the dark mode
    plt.style.use('dark_background')

    # Create the plot
    plt.figure(figsize=plot_figsize)
    plt.plot(epoch_count, train_loss, c=train_data_color, label="Training loss")
    plt.plot(epoch_count, val_loss, c=val_data_color, label="Validation loss")

    # Setting up axes
    plt.title(title, font=font_path, fontsize=16)
    plt.xlabel('X', font=font_path, fontsize=14)
    plt.ylabel('Y', font=font_path, fontsize=14)

    # Add grid and axes lines
    plt.axhline(0, color='white', lw=0.5, ls='--')
    plt.axvline(0, color='white', lw=0.5, ls='--')
    plt.grid(color='gray', linestyle='--', linewidth=0.5)

    # Customize ticks
    plt.xticks(font=font_path)
    plt.yticks(font=font_path)

    # Add a legend
    plt.legend(prop=font_path)

#### Generate example data

In [122]:
# Set the random seed for reproducibility
torch.manual_seed(42)

# Number of samples
num_samples = 32

# Generate random input features (x) and corresponding labels (y)
X = torch.linspace(0, 1, num_samples).unsqueeze(1)  # Shape (25, 1)
# Let's assume a linear relationship: y = 0.7 * x + 0.3
y = 0.7 * X + 0.3

data_df = pd.DataFrame({'X': X.numpy().flatten(), 'y': y.numpy().flatten()})

# Create train/test split
train_split = int(0.75 * len(X))  # 50% of data used for training set, 50% for testing
X_train, y_train = X[:train_split], y[:train_split]
X_val, y_val = X[train_split:], y[train_split:]

# Convert to pandas DataFrame
train_df = pd.DataFrame({'X': X_train.numpy().flatten(), 'y': y_train.numpy().flatten()})
test_df = pd.DataFrame({'X': X_val.numpy().flatten(), 'y': y_val.numpy().flatten()})

### 2. Build a model ⚒️

#### Build a model class

In [123]:
class LinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer = torch.nn.Linear(in_features=1, out_features=1)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

#### Build a training and validation loops

In [ ]:
class ConsoleColor:
    RED = '\033[91m'
    ORANGE = '\033[93m'
    GREEN = '\033[92m'
    RESET = '\033[0m'

# Initialize model, loss function, optimizer, and learning rate scheduler
model = LinearRegressionModel()
loss_function = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
learning_rate_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

# Check if GPU is available
if torch.cuda.is_available():
    print("GPUs are available!")
    print("Amount of available GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i} name:", torch.cuda.get_device_name(i))
    model.to("cuda")
else:
    print("GPU is not available. Will use CPU instead")

num_epochs = 2**7
log_interval = max(1, num_epochs // 10000)
loss_thresholds = [0.01, 0.1]  # 95% and 50% thresholds
train_loss_history = []
val_loss_history = []

for epoch in range(num_epochs):
    # Training Loop
    model.train() # Set the model to training mode

    # Compute loss on training data
    train_loss = loss_function(model(X_train), y_train)  # Assuming X_train and y_train are defined
    
    optimizer.zero_grad()  # Clear gradients
    train_loss.backward()  # Backpropagation
    optimizer.step()  # Update weights
    learning_rate_scheduler.step(train_loss.item())  # Adjust learning rate based on train loss

    # Color assignment based on loss
    if train_loss.item() < loss_thresholds[0]:
        loss_color = ConsoleColor.GREEN
    elif train_loss.item() < loss_thresholds[1]:
        loss_color = ConsoleColor.ORANGE
    else:
        loss_color = ConsoleColor.RED

    # Validation Loop
    model.eval()  # Set the model to evaluation mode
    with torch.inference_mode():  # Disable gradient tracking
        val_loss = loss_function(model(X_val), y_val)  # Assuming X_val and y_val are defined
        train_loss_history.append(train_loss.item())
        val_loss_history.append(val_loss.item())

    if epoch % log_interval == 0:
        print(f"Epoch: {epoch} ({(epoch + 1) / num_epochs * 100:.2f}% complete)"
            f" | Training Loss: {loss_color}{train_loss.item():.4f}{ConsoleColor.RESET}"
            f" | Validation Loss: {loss_color}{val_loss.item():.4f}{ConsoleColor.RESET}")

plot_loss_curves(train_loss_history, val_loss_history, range(num_epochs))

### 3. Save a model 💾

In [125]:
model_path = Path("../models/linreg.safetensors")
model_path.parent.mkdir(parents=True, exist_ok=True)

save_file(model.state_dict(), model_path)

### 4. (Re)load a model 🔃

In [126]:
tensors = {}
with safe_open(model_path, framework="pt") as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)

for name, tensor in tensors.items():
    if hasattr(model, name):
        setattr(model, name, torch.nn.Parameter(tensor))

### 5. Plot model prediction performance ✨

In [ ]:
with torch.inference_mode():
    y_preds = model(X_val)
plot_data(X_train, X_val, y_train, y_val, "Model performance", y_preds)